## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [ ]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [ ]:
# For installing the libraries & downloading models from HF Hub
!pip install huggingface_hub==0.23.2 pandas tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy -q

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [6]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Question Answering using LLM

#### Downloading and Loading the model

In [7]:
# Download and load the model
model_path = hf_hub_download(repo_id="TheBloke/Mistral-7B-Instruct-v0.2-GGUF", filename="mistral-7b-instruct-v0.2.Q6_K.gguf")

In [8]:
# Define the llm using the model_path
llm = Llama(model_path=model_path,
            n_gpu_layers=38,
            n_ctx=4096,
            n_batch=512)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#### Response

In [9]:
# Function to take the user query and generate response
def response(query,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [11]:
# User query1
query1 = "What is the protocol for managing sepsis in a critical care unit?"
response(query1)

Llama.generate: prefix-match hit


'\n\nSepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care unit:\n\n1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.\n2. Source control'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [12]:
# User query2
query2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(query2)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The symptoms of appendicitis can vary from person to person, but some common signs include:\n\n1. Abdominal pain: The pain typically starts as a mild discomfort around the navel or in the lower right abdomen. It may then move to the right lower quadrant and become more severe over time. The pain is often described as sharp, crampy, or constant.\n2. Loss of appetite: People with'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [13]:
# User query3
query3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response(query3)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can result in round or oval bald patches on the scalp, but it can also occur on other parts of the body such as the beard area, eyebrows, or eyelashes.\n\nThe exact cause of alopecia areata is not known, but it's believed to be related to a problem with the immune system. Some possible triggers for this condition include stress, genetics, viral infections, and certain medications."

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [14]:
# User query4
query4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response(query4)

Llama.generate: prefix-match hit


"\n\nA person who has sustained a physical injury to brain tissue, also known as a traumatic brain injury (TBI), may require various treatments depending on the severity and location of the injury. Here are some common treatments recommended for TBIs:\n\n1. Emergency care: The first priority is to ensure the person's airway is clear, they are breathing, and their heart is beating normally. In severe cases, emergency surgery may be required to remove hematomas or other obstructions.\n2. Medications: Depending on the symptoms, medications may be prescribed to manage conditions such as"

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [24]:
# User query5
query5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response(query5)

Llama.generate: prefix-match hit


"\n\nFirst and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:\n\n1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and prevent worsening the injury.\n2. Assess the situation: Check for any signs of shock, such as pale skin, rapid heartbeat, or shallow breathing. If you notice these symptoms, seek medical help immediately.\n3. Immobilize the leg: Use a splint, sl"

## Question Answering using LLM with Prompt Engineering

In [12]:
# System Prompt
system_prompt = """
You are a responsible AI Medical Assistant. Give clear, general medical info using trusted sources. Never diagnose or prescribe.
Always suggest seeing a doctor for personal issues. If symptoms sound serious, tell the user to get emergency help. Use simple language, respect privacy,
 admit when you don’t know.
"""

In [13]:
# Function to aggregate system prompt and user query to generate LLM response
def generate_response(query):
  global system_prompt
  prompt = f"""{system_prompt}\n
               {query}
                """
  get_response = response(prompt)
  return get_response

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [14]:
# User query1
query1 = "What is the protocol for managing sepsis in a critical care unit?"
generate_response(query1)

Llama.generate: prefix-match hit


"\nSepsis is a life-threatening condition caused by the body's response to an infection. In a critical care unit, managing sepsis involves several steps to support the patient's vital functions and address the underlying infection. Here are some general guidelines based on trusted sources like the National Institutes of Health (NIH) and the Centers for Disease Control and Prevention (CDC):\n\n1. Early recognition and diagnosis: Monitor patients closely for signs and symptoms of sepsis, such as fever, rapid heart rate, rapid breathing, confusion, or low blood pressure. Quick identification is crucial"

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [15]:
# User query2
query2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_response(query2)

Llama.generate: prefix-match hit


'\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small pouch located in the lower right side of the abdomen. Common symptoms include:\n\n1. Sudden pain that starts around the navel and then shifts to the lower right abdomen.\n2. Loss of appetite.\n3. Nausea and vomiting.\n4. Fever, which may be low-grade at first but can rise as high as 103°F (39.4°C).\n5. Abdominal swelling and rigidity.\n6.'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [16]:
# User query3
query3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_response(query3)

Llama.generate: prefix-match hit


"\nI'm an AI Medical Assistant, and I can provide some general information about sudden patchy hair loss based on trusted medical sources. However, please keep in mind that I'm not able to diagnose or prescribe treatments. It's essential to consult a healthcare professional for personal concerns regarding your health.\n\nSudden patchy hair loss, also known as alopecia areata, is an autoimmune condition where the immune system attacks hair follicles, causing hair loss in small patches on the scalp or other parts of the body. The exact cause of alopecia areata is"

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [17]:
# User query4
query4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_response(query4)

Llama.generate: prefix-match hit


"\nI'm an AI Medical Assistant and I cannot diagnose or prescribe, but I can provide some general information about the treatment options for someone with a brain injury based on trusted medical sources. Brain injuries can range from mild to severe, and treatment depends on the specific type and extent of the injury. Here are some common treatments:\n\n1. Rest and Rehabilitation: The most important thing after a brain injury is rest. This means avoiding physical or mental activities that could worsen the injury. Rehabilitation, which may include occupational therapy, speech therapy, and physical therapy, can help improve function and"

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [18]:
# User query5
query5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_response(query5)

Llama.generate: prefix-match hit


"\nI'm an AI Medical Assistant, and I cannot diagnose or prescribe, but I can provide you with general information based on trusted sources about the necessary precautions and treatment steps for someone who has fractured their leg during a hiking trip. \n\nFirstly, if you suspect that you or someone else has fractured a leg while hiking, it's essential to seek medical attention as soon as possible. In the meantime, here are some general steps you can take:\n\n1. Keep the injured leg stable and immobile by using a splint, sling, or other supportive device."

## Data Preparation for RAG

### Loading the Data

In [ ]:
# Import Google Drive library
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Path to the Input file
pdf_file_path = './data/medical_diagnosis_manual.pdf'

# Load the PDF file
loader = PyMuPDFLoader(pdf_file_path)
manual = loader.load()

### Data Overview

#### Checking the first 5 pages

In [40]:
# Checking the first 5 pages
for i, page in enumerate(manual[:5]):
    print(f"Page Number : {i+1}",end="\n")
    print(page.page_content, end="\n")

Page Number : 1
arinjoy.paul@hotmail.com
C4VK1R6FOB
ant for personal use by arinjoy.paul@hot
shing the contents in part or full is liable 

Page Number : 2
arinjoy.paul@hotmail.com
C4VK1R6FOB
This file is meant for personal use by arinjoy.paul@hotmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...............................................................................................................................................................................

#### Checking the number of pages

In [21]:
# Print number of pages
print(f"Number of pages : {len(manual)}")

Number of pages : 4114


### Data Chunking

In [22]:
# Initializing RecursiveCharacterTextSplitter
texts = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=1000,
    chunk_overlap= 200
)

In [23]:
# Splitting the document into chunks
document_chunks = loader.load_and_split(texts)

In [24]:
print(f"Number of chunks : {len(document_chunks)}")

Number of chunks : 4692


In [52]:
# Print sample chunks
for i, chunk in enumerate(document_chunks[100:103]):
    print(f"\n\nChunk Number : {i+1}",end="\n")
    print('-'*60)
    print(chunk.page_content, end="\n")



Chunk Number : 1
------------------------------------------------------------
In carotenodermia, the skin (but not the sclera) becomes deep yellow, especially on the palms and soles.
Diagnosis
• Clinical evaluation
Diagnosis is clinical. Blood vitamin levels correlate poorly with toxicity. However, if clinical diagnosis is
equivocal, laboratory testing may help. In vitamin A toxicity, fasting serum retinol levels may increase from
normal (28 to 86 μg/dL [1 to 3 μmol/L]) to > 100 μg/dL (> 3.49 μmol/L), sometimes to > 2000 μg/dL (> 69.8
μmol/L). Hypercalcemia is common.
The Merck Manual of Diagnosis & Therapy, 19th Edition Chapter 4. Vitamin Deficiency, Dependency & Toxicity
84
arinjoy.paul@hotmail.com
C4VK1R6FOB
This file is meant for personal use by arinjoy.paul@hotmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.


Chunk Number : 2
------------------------------------------------------------
Differentiating vitamin A toxicity from other di

### Embedding

In [25]:
# Initialize Embedding model
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [26]:
# Create sample embedding
sample_embedding1 = embeddings.embed_query(document_chunks[0].page_content)
sample_embedding2 = embeddings.embed_query(document_chunks[1].page_content)

In [27]:
print("Dimension of the embedding vector Sample 1: ",len(sample_embedding1))
print("Dimension of the embedding vector Sample 2: ",len(sample_embedding2))

Dimension of the embedding vector Sample 1:  384
Dimension of the embedding vector Sample 2:  384


In [57]:
# Listing one from the above sample
sample_embedding1

[-0.0800420492887497,
 0.06734584271907806,
 -0.01294492557644844,
 -0.02103177271783352,
 0.07749021798372269,
 -0.01184738427400589,
 0.08392569422721863,
 0.033526141196489334,
 -0.011201208457350731,
 0.005054276902228594,
 0.02908654883503914,
 0.03555923327803612,
 -0.04199464991688728,
 -0.029834741726517677,
 -0.046611618250608444,
 -0.04584316164255142,
 -0.0794377475976944,
 0.003304113866761327,
 -0.06368686258792877,
 0.019289657473564148,
 -0.07622833549976349,
 -0.042255643755197525,
 -0.007017737254500389,
 -0.045729655772447586,
 -0.025984320789575577,
 0.06404786556959152,
 -0.0370183140039444,
 0.07874372601509094,
 -0.021417228505015373,
 -0.07947944849729538,
 0.05655964836478233,
 -0.031771186739206314,
 0.07365147769451141,
 -0.008277401328086853,
 0.027268456295132637,
 0.051737960427999496,
 -0.07374688982963562,
 -0.09671849757432938,
 0.015409206040203571,
 -0.013661373406648636,
 0.033828288316726685,
 0.017665619030594826,
 -0.03800656273961067,
 0.079423844

### Vector Database

In [28]:
# Create ChromaDB persistent file
persist_directory = 'medicalDB'

# Create file if not present
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

In [ ]:
# Create a vector store from the embeddings and store it persistent disk
vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [61]:
# Testing Similarity search with top 3 choices
query = "What is the protocol for managing sepsis in a critical care unit?"
vector_db.similarity_search(query,k=3)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(page_content="16 - Critical Care Medicine\nChapter 222. Approach to the Critically Ill Patient\nIntroduction\nCritical care medicine specializes in caring for the most seriously ill patients. These patients are best\ntreated in an ICU staffed by experienced personnel. Some hospitals maintain separate units for special\npopulations (eg, cardiac, surgical, neurologic, pediatric, or neonatal patients). ICUs have a high\nnurse:patient ratio to provide the necessary high intensity of service, including treatment and monitoring\nof physiologic parameters.\nSupportive care for the ICU patient includes provision of adequate nutrition (see p. 21) and prevention of\ninfection, stress ulcers and gastritis (see p. 131), and pulmonary embolism (see p. 1920). Because 15 to\n25% of patients admitted to ICUs die there, physicians should know how to minimize suffering and help\ndying patients maintain dignity (see p. 3480).\nPatient Monitoring and Testing\nSome monitoring is manual (ie, by di

### Retriever

In [30]:
# Using as-retriever to perform the Similarity search
retriever = vector_db.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 3}
)

In [63]:
# Invoke the retriever
retriever.get_relevant_documents(query=query)

[Document(page_content="16 - Critical Care Medicine\nChapter 222. Approach to the Critically Ill Patient\nIntroduction\nCritical care medicine specializes in caring for the most seriously ill patients. These patients are best\ntreated in an ICU staffed by experienced personnel. Some hospitals maintain separate units for special\npopulations (eg, cardiac, surgical, neurologic, pediatric, or neonatal patients). ICUs have a high\nnurse:patient ratio to provide the necessary high intensity of service, including treatment and monitoring\nof physiologic parameters.\nSupportive care for the ICU patient includes provision of adequate nutrition (see p. 21) and prevention of\ninfection, stress ulcers and gastritis (see p. 131), and pulmonary embolism (see p. 1920). Because 15 to\n25% of patients admitted to ICUs die there, physicians should know how to minimize suffering and help\ndying patients maintain dignity (see p. 3480).\nPatient Monitoring and Testing\nSome monitoring is manual (ie, by di

### System and User Prompt Template

In [31]:
# Define the System and User prompt
qna_system_message = system_prompt

qna_user_message_template = """Use the context to answer the question.
Context: {context}
Question: {question}"""

### Response Function

In [32]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=10):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [35]:
# User query1
query1 = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(query1)

Llama.generate: prefix-match hit


'Answer: Sepsis is a serious condition that requires prompt diagnosis and treatment. In a critical care unit, the following steps are typically taken to manage sepsis:\n1. Suspect sepsis based on symptoms such as shaking chills, persistent fever, altered sensorium, hypotension, and GI symptoms (abdominal pain, nausea, vomiting, diarrhea). Septic shock develops in 25 to 40% of patients with significant bacteremia.\n2. Obtain cultures of blood and any other appropriate specimens for diagnosis.\n3'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [34]:
# User query2
query2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(query2)

Llama.generate: prefix-match hit


"Answer: The common symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia; after a few hours, the pain shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point (junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine). Additional signs include pain felt in the right lower quadrant with palpation of the left lower quadrant"

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [36]:
# User query3
query3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_rag_response(query3)

Llama.generate: prefix-match hit


'Answer: Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects people with no obvious skin or systemic disorders. The scalp and beard are most frequently affected, but any hairy area may be involved. Hair loss may affect most or all of the body (alopecia universalis). Alopecia areata is thought to be an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers.\nThe treatment options for alopecia areata include:\n1. Topical treatments: Minox'

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [37]:
# User query4
query4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(query4)

Llama.generate: prefix-match hit


'Answer: Treatments for a person with a brain injury depend on the severity and location of the damage. For traumatic brain injuries (TBIs), late seizures may develop weeks, months, or even years later, leading to motor impairments, gait and balance disturbances, ataxia, and sensory losses. If a persistent vegetative state occurs, self-awareness and other mental activity are absent, but autonomic and motor reflexes are preserved, and sleep-wake cycles are normal. Few patients recover normal neurologic function when a persistent vegetative state lasts for'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [38]:
# User query5
query5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(query5)

Llama.generate: prefix-match hit


'Answer: A fractured leg is a serious injury that requires prompt medical attention. The first step is to ensure the safety of the individual and prevent further injury. This may involve immobilizing the affected limb using a splint or sling to prevent movement and reduce pain. If the fracture is severe, hemorrhagic shock may occur, which requires immediate treatment in an emergency department.\nIn the emergency department, the fracture will be assessed and treated based on its severity. For life-threatening injuries, surgical repair of arterial injuries or nerve damage may be necessary. For'

### Fine-tuning

In [39]:
# Setting the temperature to 0.5 and regenerating the LLM response
temperature = 0.5
queries = [query1 , query2, query3, query4, query5]

# Looping through query to Re-generate the LLM response
for query in queries:
  print(f'Question: {query}')
  print(f'{generate_rag_response(query,temperature=temperature)}')
  print('-'*60)

Question: What is the protocol for managing sepsis in a critical care unit?


Llama.generate: prefix-match hit


Answer: Sepsis is a serious condition that requires prompt diagnosis and treatment. In a critical care unit, the following steps should be taken if sepsis or septic shock is suspected:
1. Obtain cultures of blood and any other appropriate specimens for laboratory testing.
2. Begin empiric antibiotics as soon as possible, preferably within 1 hour of recognition of sepsis. The choice of antibiotics should be based on the most likely pathogens and the severity of illness.
3. Provide supportive care, including fluids, antipyretics
------------------------------------------------------------
Question: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?


Llama.generate: prefix-match hit


Answer: Appendicitis is a medical condition characterized by inflammation of the appendix, which results from obstruction of its lumen. Common symptoms include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, shifting to right lower quadrant pain after several hours, increased pain with coughing or motion, and tenderness at McBurney's point. However, these classic findings appear in less than 50% of patients, as symptoms can vary greatly. If left untreated, appendicitis can lead to necrosis
------------------------------------------------------------
Question: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?


...

Answer: Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects people with no obvious skin or systemic disorder. The scalp and beard are most frequently affected, but any hairy area may be involved. Hair loss may affect most or all of the body (alopecia universalis). 
The exact cause of alopecia areata is not known, but it's believed to be an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers. 
Treatment options for al
------------------------------------------------------------
Question: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?


Llama.generate: prefix-match hit


Answer: Treatments for a person with a brain injury depend on the severity and location of the damage. For traumatic brain injuries (TBI), late seizures may develop weeks, months, or even years later, leading to motor impairments, gait and balance disturbances, ataxia, sensory losses, and in some cases, a persistent vegetative state. Speech therapists can help establish communication methods for patients with intact cognitive function. For severe injuries resulting in brain death, there is no recovery. Treatments to promote nerve regeneration are under study, including autologous macrophages
------------------------------------------------------------
Question: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?


Llama.generate: prefix-match hit


Answer: A fractured leg requires prompt medical attention. In an emergency situation, call for medical help if the injury appears severe or if the person is unable to walk. If it's safe to do so, try to immobilize the affected leg using a splint or a makeshift sling to prevent further injury and provide comfort.

Once you reach a medical facility, the healthcare professionals will evaluate the extent of the fracture and determine the appropriate treatment. Depending on the type and severity of the fracture, the person may require surgery to realign the bone or have it set in a cast
------------------------------------------------------------


## Output Evaluation

Let us now use the LLM-as-a-judge method to check the quality of the RAG system on two parameters - retrieval and generation. We illustrate this evaluation based on the answeres generated to the question from the previous section.

- We are using the same Mistral model for evaluation, so basically here the llm is rating itself on how well he has performed in the task.

In [40]:
groundedness_rater_system_message  = """
You are an impartial groundedness evaluator for a Medical Assistant chatbot. Your task is to check whether the assistant’s answer
stays fully supported by the provided trusted medical sources. Do not rely on any external knowledge — only use the given context to judge.
Highlight any parts of the answer that are not clearly supported by the sources. Rate how well the answer is grounded on a scale from 1 (not grounded) to 5
(fully grounded) and briefly explain your reasoning."""

In [41]:
relevance_rater_system_message = """
You are an impartial relevance evaluator for a Medical Assistant chatbot. Your task is to check whether the assistant’s answer directly
addresses the user’s question and stays relevant to the provided context. Do not add any new information or use outside knowledge.
Identify any parts of the answer that are off-topic or unrelated. Then rate the overall relevance on a scale from 1 (not relevant) to 5 (fully relevant)
and provide a short explanation for your rating."""

In [42]:
user_message_template = """Use the context to answer the question.
Context: {context}
Question: {question}"""

In [43]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [47]:
# User query1
query1 = "What is the protocol for managing sepsis in a critical care unit?"
ground, relevance = generate_ground_relevance_response(query1)
print(f'Grounding: \n{ground}')
print('-'*60)
print(f'Relevance: \n{relevance}')

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Grounding: 
 Based on the context provided, the answer to the question would be as follows:

Answer: If bacteremia, sepsis, or septic shock is suspected in a critically ill patient, cultures are obtained of blood and any other appropriate specimens. Empiric antibiotics are given after appropriate cultures are obtained, and early treatment appears to improve survival. Continuing therapy involves adjusting antibiotics according to the results of culture and susceptibility testing, surgically draining any abscesses, and usually removing any internal devices that are the suspected source of bacteria.

Explanation:
------------------------------------------------------------
Relevance: 
 Answer: The protocol for managing sepsis in a critical care unit includes obtaining cultures of blood and any other appropriate specimens, administering empiric antibiotics after cultures are obtained, adjusting antibiotics according to culture and susceptibility testing results, surgically draining any abs

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [49]:
# User query2
query2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
ground, relevance = generate_ground_relevance_response(query2)
print(f'Grounding: \n{ground}')
print('-'*60)
print(f'Relevance: \n{relevance}')

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Grounding: 
 Answer: The common symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, which is then followed by pain shifting to the right lower quadrant. Pain increases with cough and motion. Classic signs are direct and rebound tenderness located at McBurney's point, Rovsing sign, psoas sign, or obturator sign. Low-grade fever (37.7 to 38.3°C) is common, but these classic findings appear in less than 50% of patients.
------------------------------------------------------------
Relevance: 
 Answer: The common symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, which is then followed by pain shifting to the right lower quadrant. Pain increases with cough and motion. Classic signs are direct and rebound tenderness located at McBurney's point, Rovsing sign, psoas sign, or obturator sign. Low-grade fever (37.7 to 38.3°C) is common, but these classic findings appear

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [50]:
# User query3
query3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
ground, relevance = generate_ground_relevance_response(query3)
print(f'Grounding: \n{ground}')
print('-'*60)
print(f'Relevance: \n{relevance}')

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Grounding: 
 Based on the context provided, the question asks about treatments for sudden patchy hair loss, which is a description of alopecia areata. The sources clearly support that alopecia areata is an autoimmune disorder causing sudden patchy hair loss and that various treatment options exist for it. These include topical, intralesional, or systemic corticosteroids, topical minoxidil, topical anthralin, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA
------------------------------------------------------------
Relevance: 
 Answer: The context provided discusses various types of alopecia, including alopecia areata which is characterized by sudden patchy hair loss. According to the text, treatment options for alopecia areata include topical, intralesional, or systemic corticosteroids, topical minoxidil, topical anthralin, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA).

### Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [51]:
# User query4
query4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
ground, relevance = generate_ground_relevance_response(query4)
print(f'Grounding: \n{ground}')
print('-'*60)
print(f'Relevance: \n{relevance}')

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Grounding: 
 Based on the provided context from The Merck Manual of Diagnosis & Therapy, 19th Edition, the following treatments are mentioned for a person who has sustained a physical injury to brain tissue:

1. Speech therapists may help establish a communication code using eye blinks or movements.
2. Treatments to promote nerve regeneration are under study, including injections of autologous, incubated macrophages; epidural administration of BA-210, an experimental drug that may be neuroprotective and may stimulate nerve growth; and oral administration of HP-18
------------------------------------------------------------
Relevance: 
 Answer: The context discusses various neurological conditions following traumatic brain injuries (TBI), including decreased intellectual function, persistent vegetative state, and brain death. While some treatments are mentioned, such as speech therapy for communication and research on nerve regeneration, the answer does not directly address the question

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [52]:
# User query5
query5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
ground, relevance = generate_ground_relevance_response(query5)
print(f'Grounding: \n{ground}')
print('-'*60)
print(f'Relevance: \n{relevance}')

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Grounding: 
 Based on the context provided, the answer to the question would include the following precautions and treatment steps for a person who has fractured their leg during a hiking trip:

1. Treatment of life-threatening injuries: If there are any signs of life-threatening injuries, such as excessive bleeding or shock, prioritize these conditions and seek immediate medical attention.
2. Splinting: Immediately immobilize the fractured leg using a nonrigid or noncircumferential splint to prevent further injury and decrease pain. This can help prevent fat embolism in
------------------------------------------------------------
Relevance: 
 The user's question asks about the necessary precautions and treatment steps for someone who has fractured their leg during a hiking trip. The context provided discusses various treatments for injuries, including arterial injuries, nerve injuries, and soft tissue injuries. The answer directly addresses the user's question by discussing the initia

## Actionable Insights and Business Recommendations

####Based on the analysis and the implemented RAG system, here are some actionable insights and business recommendations:


---


**Actionable Insights:**

* **Adding Trusted Sources Improves Accuracy**: Integrating a RAG approach with trusted medical references, like the Merck Manual, clearly improves the quality of answers for specific medical questions. Compared to a base language model, this setup produces responses that are more reliable and relevant, highlighting the value of providing vetted context for medical queries.

* **Fine-Tuning Helps Adjust the Tone and Detail**: Modifying the temperature parameter has a clear impact on how responses are phrased. Keeping it low (around 0) results in more consistent and cautious answers, while a moderate setting (such as 0.5) allows for more natural variation in wording while still respecting the facts. The right setting depends on how much nuance or conservativeness is needed for different types of questions.

* **Good Context Retrieval Drives Better Grounding**: The evaluation shows that the accuracy of answers depends heavily on how closely the retrieved text matches the user’s question. This means that careful tuning of the retriever and vector store is essential to ensure that the information the system draws from stays relevant and factual.

* **A Clear System Prompt Reinforces Safe Use**: The system prompt used here does its job well by steering the model to act as a responsible assistant — it avoids stepping into diagnosis or treatment advice and always reminds users to consult a qualified doctor. This layer of guidance is vital for keeping the tool safe and trustworthy.

* **Chunking Strategy Matters**: Using a chunk size of 1,000 tokens with a 200-token overlap works well for pulling meaningful information from the Merck Manual. That said, it’s worth testing different chunk sizes and overlaps for other medical sources to find the best balance between context coverage and retrieval precision.



---


**Business Recommendations**

* **Put the RAG Tool into Daily Practice**: Roll out the RAG-based AI as an internal support tool for medical staff. It can be connected to Electronic Health Record (EHR) systems or offered through a secure web or mobile app to make it easy to access during daily workflows.

* **Broaden the Reference Material**: While the Merck Manual is an excellent foundation, expanding the knowledge base to include clinical guidelines, medical journals, and specialty textbooks will strengthen the assistant’s usefulness and depth of information.

* **Gather User Feedback Continuously**: Build in a simple way for medical staff to rate responses or flag anything unclear. This feedback loop will help keep the AI’s answers accurate and aligned with users’ needs over time.

* **Design a User-Friendly Interface**: Develop an interface that makes it easy to ask and refine questions. Features like auto-suggestions, keyword highlights in retrieved text, or the option to add more context can help clinicians get more precise answers quickly.

* **Start with High-Impact Areas**: Focus the initial rollout on specific situations where quick, reliable information is most valuable — for example, supporting emergency room teams, primary care doctors, or specialty clinics. This phased approach allows for careful testing and adjustments.

* **Prioritize Privacy and Compliance**: Medical information is highly sensitive, so any deployment must meet strict security and privacy standards like HIPAA. Make sure data is processed and stored securely and that the system is built to comply with all relevant regulations.

* **Track Use and Improve**: Keep an eye on how the system performs — monitor response times, check accuracy through periodic expert reviews, and measure how widely it’s used. These insights will highlight where refinements are needed.

* **Plan for Future Upgrades**: Once the core system is stable, look into adding new capabilities. Possibilities include accepting images (with careful disclaimers), providing concise summaries, connecting to diagnostic support tools (while clearly stating it doesn’t replace medical judgment), or customizing answers by role or specialty to make the information even more useful.

<font size=6 color='blue'>Power Ahead</font>
___